In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode_outer

import os
import sys

from typing_extensions import override

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# Initialize Spark session
spark = (SparkSession.builder.appName("DataProcessingApp")
         .config("spark.executor.memory", "10g")
         .config("spark.driver.memory", "8g")
         .getOrCreate())
spark.sparkContext.setLogLevel("DEBUG")

# Read the CSV file into a DataFrame
df = spark.read.csv('data/2019-2020_school_year/pdets.csv', header=True, inferSchema=True)

# Drop the specified columns
df = df.drop('content_source', 'tutoring_types')

# Describe the DataFrame and format the output
df.describe().show()

+-------+------------------+--------------+--------------------+--------------------+-------------------+-----------------+
|summary|        problem_id|        skills|        problem_type|student_answer_count|       mean_correct|mean_time_on_task|
+-------+------------------+--------------+--------------------+--------------------+-------------------+-----------------+
|  count|            134263|         46388|              134263|              134655|             105612|           127631|
|   mean|1185746.6052225856|          NULL|                NULL|  154.11856967806617| 0.6616545480662842|132.4985495373139|
| stddev| 556124.5188792311|          NULL|                NULL|  309.82390791199026|0.25891993108560785|139.6793436326628|
|    min|               1.0|   ['1.G.A.1']|Algebraic Expression|                   1|                0.0|            0.002|
|    max|         1869964.0|['HSS.CP.B.9']|Ungraded Open Res...|               12264|                1.0|        4721.0765|
+-------

In [2]:
from pyspark.sql.types import IntegerType

# Filter rows where 'problem_id' is not null
df = df.filter(df["problem_id"].isNotNull())

# Convert 'problem_id' to integer type
df = df.withColumn("problem_id", col("problem_id").cast(IntegerType()))

# Show the first few rows
df.show(10)

+----------+--------------------+--------------------+--------------------+-------------------+-----------------+
|problem_id|              skills|        problem_type|student_answer_count|       mean_correct|mean_time_on_task|
+----------+--------------------+--------------------+--------------------+-------------------+-----------------+
|        16|         ['8.F.B.5']|     Multiple Choice|                  16|              0.875|        62.389875|
|        33|      ['8.NS.A.2-1']|     Multiple Choice|                  80| 0.7341772151898734|             NULL|
|        35|      ['8.NS.A.2-1']|Exact Match (case...|                  34| 0.3235294117647059|             NULL|
|        37|      ['8.NS.A.2-1']|Exact Match (case...|                  28| 0.8571428571428571|             NULL|
|        39|      ['8.NS.A.2-1']|     Multiple Choice|                  26| 0.4230769230769231|             NULL|
|        48|     ['5.NF.B.4a-1']|     Multiple Choice|                   6|0.16666666666

In [3]:
from pyspark.sql.functions import isnan, when, count
# df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+----------+------+------------+--------------------+------------+-----------------+
|problem_id|skills|problem_type|student_answer_count|mean_correct|mean_time_on_task|
+----------+------+------------+--------------------+------------+-----------------+
|         0| 87875|           0|                   0|       28953|             7024|
+----------+------+------------+--------------------+------------+-----------------+



In [4]:
from pyspark.sql.functions import col, explode, split, regexp_replace, array, size, lit

df = df.withColumn("skills", when(col("skills").isNull(), "[]").otherwise(col("skills")))

# Clean the skills string and convert to array
# Note: we need to handle the case where cleaning results in empty string
df = df.withColumn("skills", 
    regexp_replace(regexp_replace(col("skills"), r"[\[\]'\s]", ""), r",,", ","))
df = df.withColumn("skills_array", when(
    col("skills") == "", array().cast("array<string>")
).otherwise(
    split(col("skills"), ",")
))

# Explode the array to create one row per skill
df = df.withColumn("skill", explode_outer(col("skills_array")))

# Extract grade, domain, and subdomain
df = df.withColumn("parts", split(col("skill"), "\\."))
new_df = df.select(
    col("problem_id"),
    col("skills"),
    col("problem_type"),
    col("student_answer_count"),
    col("mean_correct"),
    col("mean_time_on_task"),
    col("skill").alias("skill_code"),
    col("parts").getItem(0).alias("grade"),
    col("parts").getItem(1).alias("domain"),
    col("parts").getItem(2).alias("subdomain")
)

# Cache the DataFrame
new_df = new_df.cache()

# Show the first few rows
new_df.show()

+----------+--------------------+--------------------+--------------------+-------------------+-----------------+-----------+-----+------+---------+
|problem_id|              skills|        problem_type|student_answer_count|       mean_correct|mean_time_on_task| skill_code|grade|domain|subdomain|
+----------+--------------------+--------------------+--------------------+-------------------+-----------------+-----------+-----+------+---------+
|        16|             8.F.B.5|     Multiple Choice|                  16|              0.875|        62.389875|    8.F.B.5|    8|     F|        B|
|        33|          8.NS.A.2-1|     Multiple Choice|                  80| 0.7341772151898734|             NULL| 8.NS.A.2-1|    8|    NS|        A|
|        35|          8.NS.A.2-1|Exact Match (case...|                  34| 0.3235294117647059|             NULL| 8.NS.A.2-1|    8|    NS|        A|
|        37|          8.NS.A.2-1|Exact Match (case...|                  28| 0.8571428571428571|           

In [5]:
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+----------+------+------------+--------------------+------------+-----------------+------------+-----+-----+
|problem_id|skills|problem_type|student_answer_count|mean_correct|mean_time_on_task|skills_array|skill|parts|
+----------+------+------------+--------------------+------------+-----------------+------------+-----+-----+
|         0|     0|           0|                   0|       30883|             7707|           0|87875|87875|
+----------+------+------------+--------------------+------------+-----------------+------------+-----+-----+



In [6]:
df.count()

141495

In [7]:
new_df.count()

141495

In [26]:
plogs = spark.read.csv('data/2019-2020_school_year/plogs.csv', header=True, inferSchema=True)
print(plogs.count())

plogs = plogs.select('assignment_id', 'problem_id').distinct()
plogs.filter(plogs.assignment_id == 14101).select('assignment_id', 'problem_id').show()
plogs.show(10)

20752836
+-------------+----------+
|assignment_id|problem_id|
+-------------+----------+
|        14101|     43718|
|        14101|     44022|
|        14101|     43683|
+-------------+----------+

+-------------+----------+
|assignment_id|problem_id|
+-------------+----------+
|        14000|   1322054|
|        14138|   1469127|
|        14040|    658671|
|        14180|   1088882|
|        14004|   1618042|
|        14196|   1520113|
|        13867|   1719799|
|        13862|    896098|
|        13866|   1719285|
|        13707|   1469212|
+-------------+----------+
only showing top 10 rows



In [27]:
new_df.filter(new_df.problem_id == 44022).show()

+----------+------+---------------+--------------------+------------+-----------------+----------+-----+------+---------+
|problem_id|skills|   problem_type|student_answer_count|mean_correct|mean_time_on_task|skill_code|grade|domain|subdomain|
+----------+------+---------------+--------------------+------------+-----------------+----------+-----+------+---------+
|     44022|      |Multiple Choice|                  29|         1.0|          262.763|      NULL| NULL|  NULL|     NULL|
+----------+------+---------------+--------------------+------------+-----------------+----------+-----+------+---------+



In [29]:
new_df.filter(new_df.problem_id == 43683).show()

+----------+------+---------------+--------------------+------------------+-----------------+----------+-----+------+---------+
|problem_id|skills|   problem_type|student_answer_count|      mean_correct|mean_time_on_task|skill_code|grade|domain|subdomain|
+----------+------+---------------+--------------------+------------------+-----------------+----------+-----+------+---------+
|     43683|      |Multiple Choice|                  29|0.4482758620689655| 61.9871379310345|      NULL| NULL|  NULL|     NULL|
+----------+------+---------------+--------------------+------------------+-----------------+----------+-----+------+---------+



In [9]:
from pyspark.sql import Window
from pyspark.sql.functions import count, desc, row_number

# Filter out rows where subdomain is null, and collect rows needing processing
rows_to_process = new_df.where(new_df.subdomain.isNull()).select('problem_id')
rows_to_process = rows_to_process.withColumnRenamed('problem_id', 'no_skill_problem_id')
no_skill_in_assignments = rows_to_process.join(plogs, rows_to_process.no_skill_problem_id == plogs.problem_id, 'left').drop('problem_id')

new_df_has_skills = new_df.where(new_df.subdomain.isNotNull())
plogs_has_skills = new_df_has_skills.join(plogs, plogs.problem_id == new_df_has_skills.problem_id, 'left').select('assignment_id', 'grade', 'domain', 'subdomain')

no_skill_impute_data = no_skill_in_assignments.join(plogs_has_skills, no_skill_in_assignments.assignment_id == plogs_has_skills.assignment_id, 'left').drop('assignment_id')

# Define a window partitioned by no_skill_problem_id and ordered by frequency in descending order
window = Window.partitionBy("no_skill_problem_id").orderBy(desc("count"))

# Count occurrences of each grade, domain, and subdomain per no_skill_problem_id
df_counts = no_skill_impute_data.groupBy("no_skill_problem_id", "grade", "domain", "subdomain").count()

# Get the most frequent grade, domain, and subdomain for each no_skill_problem_id
result = df_counts.withColumn("rank", row_number().over(window)) \
                  .filter(col("rank") == 1) \
                  .select("no_skill_problem_id", "grade", "domain", "subdomain")


In [13]:
result = result.withColumnsRenamed({'grade': 'impute_grade', 'domain': 'impute_domain', 'subdomain': 'impute_subdomain'})

In [18]:
result.select([count(when(col(c).isNull(), c)).alias(c) for c in result.columns]).show()

+-------------------+------------+-------------+----------------+
|no_skill_problem_id|impute_grade|impute_domain|impute_subdomain|
+-------------------+------------+-------------+----------------+
|                  0|       76652|        76652|           76652|
+-------------------+------------+-------------+----------------+



In [19]:
result.count()

87875

In [14]:
from pyspark.sql.functions import coalesce

df_updated = new_df.join(result, new_df.problem_id == result.no_skill_problem_id, how='left').select(
    *[new_df[col] for col in new_df.columns if col not in ['grade', 'domain', 'subdomain']],  # Select all columns except E and F
    coalesce(result['impute_grade'], new_df['grade']).alias('grade'),  # Replace E with result.E if available
    coalesce(result['impute_domain'], new_df['domain']).alias('domain'),   # Replace F with result.F if available
    coalesce(result['impute_subdomain'], new_df['subdomain']).alias('subdomain') 
)
df_updated.show(10)

+----------+--------------------+--------------------+--------------------+-------------------+-----------------+-----------+-----+------+---------+
|problem_id|              skills|        problem_type|student_answer_count|       mean_correct|mean_time_on_task| skill_code|grade|domain|subdomain|
+----------+--------------------+--------------------+--------------------+-------------------+-----------------+-----------+-----+------+---------+
|        16|             8.F.B.5|     Multiple Choice|                  16|              0.875|        62.389875|    8.F.B.5|    8|     F|        B|
|        48|         5.NF.B.4a-1|     Multiple Choice|                   6|0.16666666666666666|             NULL|5.NF.B.4a-1|    5|    NF|        B|
|        37|          8.NS.A.2-1|Exact Match (case...|                  28| 0.8571428571428571|             NULL| 8.NS.A.2-1|    8|    NS|        A|
|        61|           7.G.B.5-2|Exact Match (case...|                  28| 0.6428571428571429| 398.008142

In [15]:
df_updated.count()

141495

In [20]:
df_updated = df_updated.drop('skills', 'skill_code')
df_updated.select([count(when(col(c).isNull(), c)).alias(c) for c in df_updated.columns]).show()

+----------+------------+--------------------+------------+-----------------+-----+------+---------+
|problem_id|problem_type|student_answer_count|mean_correct|mean_time_on_task|grade|domain|subdomain|
+----------+------------+--------------------+------------+-----------------+-----+------+---------+
|         0|           0|                   0|       30883|             7707|76652| 76652|    76652|
+----------+------------+--------------------+------------+-----------------+-----+------+---------+



In [22]:
df_updated.filter(df_updated.grade.isNull()).orderBy('problem_id').show()

+----------+--------------------+--------------------+-------------------+-----------------+-----+------+---------+
|problem_id|        problem_type|student_answer_count|       mean_correct|mean_time_on_task|grade|domain|subdomain|
+----------+--------------------+--------------------+-------------------+-----------------+-----+------+---------+
|     43683|     Multiple Choice|                  29| 0.4482758620689655| 61.9871379310345| NULL|  NULL|     NULL|
|     43718|     Multiple Choice|                  29| 0.7241379310344828| 29.3214137931034| NULL|  NULL|     NULL|
|     44022|     Multiple Choice|                  29|                1.0|          262.763| NULL|  NULL|     NULL|
|     61089|Algebraic Expression|                 120| 0.7226890756302521| 111.104512605042| NULL|  NULL|     NULL|
|     61090|Algebraic Expression|                 112| 0.5137614678899083| 133.204770642202| NULL|  NULL|     NULL|
|     61092|Algebraic Expression|                   1|                1.

In [32]:
df_updated.filter(df_updated.grade.isNull()).count() 

76652